# LiDAR LAZ Tiling with LAStools

This notebook‑style script shows how to split large LiDAR `.laz` files into smaller, overlapping tiles using the LAStools `lastile` utility. Each section includes explanations and runnable Python code.

---

## Prerequisites

- **Python 3.x**  
- **LAStools** installed, with `lastile` on your system PATH  
- A folder of input `.laz` files to tile  

Run this cell to verify `lastile` is available:


In [3]:
import os
import subprocess

In [4]:
try:
    subprocess.run(["lastile", "-h"], stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
    print(" lastile is available!")
except FileNotFoundError:
    print(" lastile not found. Please install LAStools and add to PATH.")

 lastile is available!


## Configuration & tiling workflow

This script chops up reprojected LAZ files into square tiles (with a buffer) using LAStools’ `lastile`.
Tiles come out as compressed `.laz` and are ready for the next step in the pipeline.

### Key settings

* **INPUT\_DIR** – folder with your reprojected `.laz` files.
* **OUTPUT\_DIR** – where the tiles will go. If it doesn’t exist, the script makes it.
* **TILE\_SIZE** – tile width/height in meters (e.g., 1000 for 1×1 km tiles).
* **BUFFER** – extra overlap (in meters) around each tile so features at the edges don’t get clipped.
* **CORES** – number of CPU cores for `lastile` to use. More cores = faster tiling.
* **OUTPUT\_FORMAT** – either `-olaz` (compressed) or `-olas` (uncompressed LAS).

---

### What the script does

1. **Create the output folder**

   ```python
   os.makedirs(OUTPUT_DIR, exist_ok=True)
   ```

   No need to manually make it — it’ll be ready for writing tiles.

2. **Loop through input files**

   ```python
   for filename in os.listdir(INPUT_DIR):
       if not filename.lower().endswith(".laz"):
           continue
       input_path = os.path.join(INPUT_DIR, filename)
   ```

   Skips anything that’s not `.laz` and grabs the full file path for each input.

3. **Build the `lastile` command**

   ```python
   command = [
       "lastile",
       "-i", input_path,
       "-tile_size", str(TILE_SIZE),
       "-buffer", str(BUFFER),
       "-odir", OUTPUT_DIR,
       "-olaz",
       "-cores", str(CORES)
   ]
   ```

   Main options:

   * `-tile_size` → tile dimensions
   * `-buffer` → overlap in meters
   * `-olaz` or `-olas` → compression setting
   * `-cores` → run in parallel

4. **Run it and keep going on errors**
   If `lastile` fails on one file, the script catches it and moves on. That way one bad LAZ doesn’t kill the whole run.

---

### If you’d rather do it by hand

You can skip the Python wrapper and just run `lastile` in your terminal:

```bash
lastile \
  -i "C:/…/USFS_Tahoe_National_2014_reproj/*.laz" \
  -tile_size 1000 \
  -buffer 20 \
  -odir "C:/…/USFS_Tahoe_National_Tiled_2014" \
  -olaz \
  -cores 4
```

In [5]:
import os
import subprocess

# Path to folder containing your input .laz files
INPUT_DIR = r"C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer"

# Path where the tiled files will be written (will be created if needed)
OUTPUT_DIR = r"C:\Users\sreeja\Documents\Kaibab\Mangum_Tiled"

# Size of each square tile in meters
TILE_SIZE = 1000

# Buffer (overlap) around each tile in meters, to avoid edge effects
BUFFER = 20

# Number of CPU cores to use for parallel tiling
CORES = 4

# Output format: 'laz' (compressed) or 'las' (uncompressed)
OUTPUT_FORMAT = "laz"

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
print(f"Output directory ready: {OUTPUT_DIR}")

Output directory ready: C:\Users\sreeja\Documents\Kaibab\Mangum_Tiled


In [7]:
for filename in os.listdir(INPUT_DIR):
    if not filename.lower().endswith(".laz"):
        continue

    input_path = os.path.join(INPUT_DIR, filename)
    print(f"Processing {input_path}…")

    command = [
        "lastile",
        "-i",         input_path,        # input file
        "-tile_size", str(TILE_SIZE),     # tile size in meters
        "-buffer",    str(BUFFER),        # buffer in meters
        "-odir",      OUTPUT_DIR,         # output directory
        "-olaz",                           # write compressed LASzip (.laz)
        "-cores",     str(CORES)          # number of parallel cores
    ]

    try:
        subprocess.run(command, check=True)
        print(f" Finished tiling {filename}")
    except subprocess.CalledProcessError as e:
        print(f" Error tiling {filename}: exit code {e.returncode}")
        continue

print(f"\nAll done! Tiles are in:\n   {OUTPUT_DIR}")

Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E2101_Mangum2021.laz…
 Finished tiling q36112E2101_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E2102_Mangum2021.laz…
 Finished tiling q36112E2102_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E2103_Mangum2021.laz…
 Finished tiling q36112E2103_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E3102_Mangum2021.laz…
 Finished tiling q36112E3102_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E3103_Mangum2021.laz…
 Finished tiling q36112E3103_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E3104_Mangum2021.laz…
 Finished tiling q36112E3104_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E3106_Mangum2021.laz…
 Finished tiling q36112E3106_Mangum2021.laz
Processing C:\Users\sreeja\Documents\Kaibab\Mangum_No_Buffer\q36112E3

KeyboardInterrupt: 